In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_majority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_minority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_minority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_minority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_majority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_majority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_minority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_majority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_dev.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_gold_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_gold_soft.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_gold_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_gold_hard.json


In [4]:
pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.1/606.1 kB 9.6 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
Note: you may need to restart the kernel to use updated packages.


In [8]:
import wandb

wandb.login(key="0c5f368f1f51fd942ec7bb3a1c74efb7bdc832d6")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mshoaibvohra (mshoaibvohra-habib-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.1 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#this is english to spanish

import json
import torch
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm  # Progress bar

# Define the model name for Spanish to English translation.
model_name = "Helsinki-NLP/opus-mt-en-es"

# Load the tokenizer and model.
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Check if GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate_text(text):
    """Translate Spanish text to English using Helsinki-NLP/opus-mt-en-es."""
    # Tokenize input text and move tensors to GPU if available.
    batch = tokenizer([text], return_tensors="pt", padding=True)
    batch = {k: v.to(device) for k, v in batch.items()}
    
    # Generate translation
    with torch.no_grad():
        translated = model.generate(**batch)
    
    # Decode and return translated text.
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Load the JSON file.
file_path = "/kaggle/input/exist2025-all/EXIST2025_training.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Count Spanish tweets
total_spanish_tweets = sum(1 for v in data.values() if v["lang"] == "en")
translated_count = 0

# Translate Spanish tweets and update the split value.
for key, value in tqdm(data.items(), desc="Translating tweets", total=total_spanish_tweets):
    if value["lang"] == "en":
        value["tweet"] = translate_text(value["tweet"])
        value["split"] = "TRAIN_ES"
        translated_count += 1

        # Print progress every 100 tweets
        if translated_count % 100 == 0:
            print(f"Translated {translated_count}/{total_spanish_tweets} tweets")

# Save the updated JSON file.
updated_file_path = "EXIST2025_training_translated.json"
with open(updated_file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"\n✅ Translation complete. {translated_count}/{total_spanish_tweets} tweets translated.")
print(f"File saved as: {updated_file_path}")


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

...

Translating tweets: 5060it [11:48,  2.10it/s]

Translated 1400/3260 tweets


Translating tweets: 5160it [12:39,  2.42it/s]

Translated 1500/3260 tweets


Translating tweets: 5239it [13:12,  2.48it/s]

In [ ]:
#this is spanish to english


import json
import torch
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm  # Progress bar

# Define the model name for Spanish to English translation.
model_name = "Helsinki-NLP/opus-mt-es-en"

# Load the tokenizer and model.
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Check if GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate_text(text):
    """Translate Spanish text to English using Helsinki-NLP/opus-mt-es-en."""
    # Tokenize input text and move tensors to GPU if available.
    batch = tokenizer([text], return_tensors="pt", padding=True)
    batch = {k: v.to(device) for k, v in batch.items()}
    
    # Generate translation
    with torch.no_grad():
        translated = model.generate(**batch)
    
    # Decode and return translated text.
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Load the JSON file.
file_path = "/content/EXIST2025_training.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Count Spanish tweets
total_spanish_tweets = sum(1 for v in data.values() if v["lang"] == "es")
translated_count = 0

# Translate Spanish tweets and update the split value.
for key, value in tqdm(data.items(), desc="Translating tweets", total=total_spanish_tweets):
    if value["lang"] == "es":
        value["tweet"] = translate_text(value["tweet"])
        value["split"] = "TRAIN_EN"
        translated_count += 1

        # Print progress every 100 tweets
        if translated_count % 100 == 0:
            print(f"Translated {translated_count}/{total_spanish_tweets} tweets")

# Save the updated JSON file.
updated_file_path = "EXIST2025_training_translated.json"
with open(updated_file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"\n✅ Translation complete. {translated_count}/{total_spanish_tweets} tweets translated.")
print(f"File saved as: {updated_file_path}")
